In [77]:
import csv
import math

# TODO remove when not on google colab
from google.colab import files
uploaded = files.upload()

Saving dadosBenchmark_validacaoAlgoritmoAD.csv to dadosBenchmark_validacaoAlgoritmoAD.csv


In [80]:
import random

DISCRETE_ATT = 1
CONTINUOUS_ATT = 2

dataset = []
preditiveAttributes = {}
allAttributes = {}
attributesTypes = {}
EXPECTED_COLUMN = ""

def openFile(filename, delimiter, attTypes):
  with open(filename, 'r') as file:
    global attributesTypes, preditiveAttributes, allAttributes, dataset, EXPECTED_COLUMN
    lines = csv.reader(file, delimiter=delimiter)
    list_lines = list(lines)

    attributesTypes = attTypes
    preditiveAttributes = {x:i for i,x in enumerate(list_lines[0][:-1])}
    allAttributes = {x:i for i,x in enumerate(list_lines[0])} #Includes Joga

    dataset = list_lines[1:]
    EXPECTED_COLUMN = list_lines[0][-1]

    
#openFile("diabetes.csv", ',', {'Pregnancies': CONTINUOUS_ATT, 'Glucose': CONTINUOUS_ATT, 'BloodPressure': CONTINUOUS_ATT, 'SkinThickness': CONTINUOUS_ATT, 'Insulin': CONTINUOUS_ATT, 'BMI': CONTINUOUS_ATT, 'DiabetesPedigreeFunction': CONTINUOUS_ATT, 'Age': CONTINUOUS_ATT, 'Outcome': DISCRETE_ATT})
openFile("dadosBenchmark_validacaoAlgoritmoAD.csv", ';', {'Temperatura': DISCRETE_ATT, 'Tempo': DISCRETE_ATT, 'Umidade': DISCRETE_ATT, 'Ventoso': DISCRETE_ATT, 'Joga': DISCRETE_ATT})

print("dataset: " + str(dataset))
print("allAttributes:" + str(allAttributes))
print("preditiveAttributes:" + str(preditiveAttributes))
print("attributeTypes:" + str(attributesTypes))


dataset: [['Ensolarado', 'Quente', 'Alta', 'Falso', 'Nao'], ['Ensolarado', 'Quente', 'Alta', 'Verdadeiro', 'Nao'], ['Nublado', 'Quente', 'Alta', 'Falso', 'Sim'], ['Chuvoso', 'Amena', 'Alta', 'Falso', 'Sim'], ['Chuvoso', 'Fria', 'Normal', 'Falso', 'Sim'], ['Chuvoso', 'Fria', 'Normal', 'Verdadeiro', 'Nao'], ['Nublado', 'Fria', 'Normal', 'Verdadeiro', 'Sim'], ['Ensolarado', 'Amena', 'Alta', 'Falso', 'Nao'], ['Ensolarado', 'Fria', 'Normal', 'Falso', 'Sim'], ['Chuvoso', 'Amena', 'Normal', 'Falso', 'Sim'], ['Ensolarado', 'Amena', 'Normal', 'Verdadeiro', 'Sim'], ['Nublado', 'Amena', 'Alta', 'Verdadeiro', 'Sim'], ['Nublado', 'Quente', 'Normal', 'Falso', 'Sim'], ['Chuvoso', 'Amena', 'Alta', 'Verdadeiro', 'Nao']]
allAttributes:{'Tempo': 0, 'Temperatura': 1, 'Umidade': 2, 'Ventoso': 3, 'Joga': 4}
preditiveAttributes:{'Tempo': 0, 'Temperatura': 1, 'Umidade': 2, 'Ventoso': 3}
attributeTypes:{'Temperatura': 1, 'Tempo': 1, 'Umidade': 1, 'Ventoso': 1, 'Joga': 1}


In [81]:
# Calculate entropy for given attribute
def entropy(dataset, attribute_name):
  
  attribute = allAttributes[attribute_name]
  
  frequency = {}
  attribute_entropy = 0;
  
  for line in dataset:
    if line[attribute] in frequency:
      frequency[line[attribute]] += 1.0
    else:
      frequency[line[attribute]] = 1.0
            
  for value in frequency.values():
    pi = value / len(dataset)
    attribute_entropy += - pi * math.log(pi,2)
  
  return round(attribute_entropy,5)

for attribute in allAttributes:
  print("Entropia para " + attribute + ": " + str(entropy(dataset, attribute)))


Entropia para Tempo: 1.57741
Entropia para Temperatura: 1.55666
Entropia para Umidade: 1.0
Entropia para Ventoso: 0.98523
Entropia para Joga: 0.94029


In [82]:

# Calculate gain for a given attribute
def gain(dataset, attribute_name):
  frequency = {} # frequency of a value 
  total_gain = entropy(dataset, EXPECTED_COLUMN) # entropy of default

  attribute = preditiveAttributes[attribute_name]
  
  for line in dataset:
    if line[attribute] in frequency:
      frequency[line[attribute]] += 1.0
    else:
      frequency[line[attribute]] = 1.0
      
  for k, v in frequency.items():
    pi = v / len(dataset)
    subset = [line for line in dataset if line[attribute] == k]
    total_gain -= pi * entropy(subset, EXPECTED_COLUMN)
      
  return round(total_gain,3)



for attribute in preditiveAttributes:
  print("Ganho para " + attribute + ": " + str(gain(dataset, attribute)))



Ganho para Tempo: 0.247
Ganho para Temperatura: 0.029
Ganho para Umidade: 0.152
Ganho para Ventoso: 0.048


In [0]:
#@title Define Tree Class
import operator

class Node(object):
  def __init__(self, attribute=None):
    self.attribute = attribute
    self.classType = None
    self.operator = operator.eq
    self.descendents = {}
    
  def addBranch(self, subAttribute, node):
    #print("addBranch: adicionando para " + str(subAttribute) + str(op) + str(node))
    print("addBranch: adicionando para " + str(subAttribute) + str(node))
    self.descendents[subAttribute] = node
    print("Nodo " + str(self) + " possui descendentes: " + str(self.descendents))
    #self.operator = op
    #self.descendents[subAttribute].updateOperator(op)
    
  def updateClass(self, classType):
    self.classType = classType

  def updateAttribute(self, attribute):
    self.attribute = attribute
    
  def updateOperator(self, op):
    self.operator = op


In [93]:
def countClasses(dataset, attribute_name=EXPECTED_COLUMN):
  frequency = {} # frequency of a value

  attribute = allAttributes[attribute_name]
  
  for line in dataset:
    if line[attribute] in frequency:
      frequency[line[attribute]] += 1.0
    else:
      frequency[line[attribute]] = 1.0
      
  return frequency
      
countClasses(dataset)

{'Nao': 5.0, 'Sim': 9.0}

In [0]:
def decisionTree(dataset, attributes):
  #print("decisionTree() chamada para dataset e atributos: " + str(attributes))
  # Crie um nó N
  node = Node()

  datasetClasses = countClasses(dataset)
  
  #print("decisionTree() attributes: " + str(attributes))

  # Se todos os exemplos em D possuem a mesma classe yi, então retorne N como um nó folha rotulado com y
  if len(datasetClasses) == 1:
    
    #print("Todos exemplos com mesma classe " + str(dataset) + str(attributes))
    node.updateClass(list(countClasses(dataset).keys())[0])
    return node

  # Se L é vazia, então retorne N como um nó folha rotulado com a classe yi mais frequente em D
  elif len(attributes) == 0:

    #print("L vazia")
    higherFrequency = -1

    for key in datasetClasses:
      if datasetClasses[key] > higherFrequency:
        higherFrequency = datasetClasses[key]

    node.updateClass(list(countClasses(dataset).keys())[0])
    return node

  else:
    #print("Achar melhor criterio de divisão " + str(attributes))
    # A = atributo preditivo em L que apresenta "melhor" critério de divisão
    bestAttribute = random.sample(attributes.keys(), 1)[0]
    bestAttributeGain = 0
    randomAttributes = random.sample(attributes.keys(), min(int(math.sqrt(len(preditiveAttributes))), len(attributes)))
    #print("random attributes= " + str(randomAttributes))
    for attribute in randomAttributes:
      currentAttributeGain = gain(dataset, attribute)
      if currentAttributeGain > bestAttributeGain:
        bestAttribute = attribute
        bestAttributeGain = currentAttributeGain
        #print("-> new best attribute: " + str(attribute))
        #print("-> new gain: " + str(gain(dataset, attribute)))

    # Associe A ao nó N
    node.updateAttribute(bestAttribute)

    # L = L - A
    del attributes[bestAttribute]

    #Para cada valor v distinto do atributo A, considerando os exemplos em D, faça
    comparableAttributes = []       # lista com nome do atributo e operador
    if attributesTypes[bestAttribute] == DISCRETE_ATT:
      for i in countClasses(dataset, bestAttribute).keys():
        comparableAttributes.append([i, operator.eq])
      print(str(comparableAttributes))
    else :
      average = sum(float(line[preditiveAttributes[bestAttribute]]) for line in dataset) / len(dataset)
      comparableAttributes.append([average, operator.le])
      comparableAttributes.append([average, operator.gt])
    
    for subAttribute in comparableAttributes: 
      print(str(subAttribute))
      #Dv = subconjunto dos dados de treinamento em que A = v
      subDataset = []
      
      value, op = subAttribute
      
      for line in dataset:
        if op == operator.eq: # DISCRETE_ATTRIBUTE
          if line[preditiveAttributes[bestAttribute]] == value:
            subDataset.append(line)  
        else: # CONTINUOUS ATTRIBUTE
          if op(float(line[preditiveAttributes[bestAttribute]]), value):
            #print("Average: " + str(value) + " - number: " + str(line[preditiveAttributes[bestAttribute]]))
            subDataset.append(line)
      
      #Se Dv vazio, então retorne N como um nó folha rotulado com a classe yi mais frequente em Dv. 
      if len(subDataset) == 0:
        print("Dv vazio")
        subDatasetClasses = countClasses(subDataset)
        
        higherFrequency = -1

        for key in subDatasetClasses:
          if subDatasetClasses[key] > higherFrequency:
            higherFrequency = subDatasetClasses[key]

        node.updateClass(list(countClasses(dataset).keys())[0])
        #return node
      
      #Senão, associe N a uma subárvore retornada por arvoreDeDecisao(Dv,L)    
      else: 
        print("\n")
        print(value)
        print(op)
        node.addBranch(value, decisionTree(subDataset, attributes))
        #newnode = node.addBranch(value, op, decisionTree(subDataset, attributes))
        #newnode = decisionTree(subDataset, attributes)
        
    #retorne N
    return node
      
      
  
  

In [119]:
import copy

copyPreditiveAttributes = copy.copy(preditiveAttributes)
root_node = decisionTree(dataset, copyPreditiveAttributes)


[['Ensolarado', <built-in function eq>], ['Nublado', <built-in function eq>], ['Chuvoso', <built-in function eq>]]
['Ensolarado', <built-in function eq>]


Ensolarado
<built-in function eq>
[['Alta', <built-in function eq>], ['Normal', <built-in function eq>]]
['Alta', <built-in function eq>]


Alta
<built-in function eq>
addBranch: adicionando para Alta<__main__.Node object at 0x7f1b6c4d7160>
Nodo <__main__.Node object at 0x7f1b6c4d70f0> possui descendentes: {'Alta': <__main__.Node object at 0x7f1b6c4d7160>}
['Normal', <built-in function eq>]


Normal
<built-in function eq>
addBranch: adicionando para Normal<__main__.Node object at 0x7f1b6c4d71d0>
Nodo <__main__.Node object at 0x7f1b6c4d70f0> possui descendentes: {'Alta': <__main__.Node object at 0x7f1b6c4d7160>, 'Normal': <__main__.Node object at 0x7f1b6c4d71d0>}
addBranch: adicionando para Ensolarado<__main__.Node object at 0x7f1b6c4d70f0>
Nodo <__main__.Node object at 0x7f1b6c4d7080> possui descendentes: {'Ensolarado': <__main__.No

In [120]:
def printTree(node, level=0):
  print(''.join(['   ' for s in range(level)]) + str(node.attribute) + ": " + str(node.classType))
  for k, v in node.descendents.items():
    print(''.join(['   ' for s in range(level)]) + '-> ' + str(k))
    printTree(v, level + 1)
    
printTree(root_node)
print(str(root_node))
print(str(root_node.descendents))

Tempo: None
-> Ensolarado
   Umidade: None
   -> Alta
      None: Nao
   -> Normal
      None: Sim
-> Nublado
   None: Sim
-> Chuvoso
   Ventoso: None
   -> Falso
      None: Sim
   -> Verdadeiro
      None: Nao
{'Ensolarado': <__main__.Node object at 0x7f1b6c4d70f0>, 'Nublado': <__main__.Node object at 0x7f1b6c4d7128>, 'Chuvoso': <__main__.Node object at 0x7f1b6c4d7278>}


In [0]:
def classify(node, entry):
  
  if not node.descendents:
    return node.classType
  
  #print(str(node.descendents.keys()))
  #return

  for value, desc_node in node.descendents.items():
    #print(value)
    #print(desc_node)
    #return 
    if value == entry[preditiveAttributes[node.attribute]]:
      return classify(desc_node, entry)
  
  return None
  

In [122]:
example = dataset[10]
print(example)
example = dataset[10][:-1]

print("Resultado: " + str(classify(root_node, example)))
#TODO as vezes ta retornando None pq a arvore foi montada com um conjunto q nao tinha algum tipo de atributo. O que se faz, se descarta none?

['Ensolarado', 'Amena', 'Normal', 'Verdadeiro', 'Sim']
Resultado: Sim


In [0]:
ntree = 10
import random

def createBootstrap(dataset, fraction):
  bootstrap = random.sample(dataset, int(len(dataset)*fraction))
  bootstrap += (random.sample(bootstrap, len(dataset) - len(bootstrap)))
  
  #TODO precisa criar testing set?
  return bootstrap

In [124]:

def createForest():
  forest = []
  for i in range(ntree):
    bootstrap = createBootstrap(dataset, fraction=0.66)
    copyPreditiveAttributes = copy.copy(preditiveAttributes)
    tree = decisionTree(bootstrap, copyPreditiveAttributes)
    forest.append(tree)
  
  return forest

def most_common(lst):
    return max(set(lst), key=lst.count)

def main():
    forest = createForest();
    
    testSet = [dataset[5][:-1], dataset[10][:-1]]
    for test in testSet:
      results = []
      for tree in forest:
        answer = classify(tree, test)
        if answer != None:
          results.append(answer)
      
      print(results)
      print("Votação majoritária: " + str(most_common(results)))

main()

[['Falso', <built-in function eq>], ['Verdadeiro', <built-in function eq>]]
['Falso', <built-in function eq>]


Falso
<built-in function eq>
[['Ensolarado', <built-in function eq>], ['Nublado', <built-in function eq>], ['Chuvoso', <built-in function eq>]]
['Ensolarado', <built-in function eq>]


Ensolarado
<built-in function eq>
addBranch: adicionando para Ensolarado<__main__.Node object at 0x7f1b6c4c47f0>
Nodo <__main__.Node object at 0x7f1b6c4c4860> possui descendentes: {'Ensolarado': <__main__.Node object at 0x7f1b6c4c47f0>}
['Nublado', <built-in function eq>]


Nublado
<built-in function eq>
addBranch: adicionando para Nublado<__main__.Node object at 0x7f1b6c4d11d0>
Nodo <__main__.Node object at 0x7f1b6c4c4860> possui descendentes: {'Ensolarado': <__main__.Node object at 0x7f1b6c4c47f0>, 'Nublado': <__main__.Node object at 0x7f1b6c4d11d0>}
['Chuvoso', <built-in function eq>]


Chuvoso
<built-in function eq>
addBranch: adicionando para Chuvoso<__main__.Node object at 0x7f1b6c4d1198>